<a href="https://colab.research.google.com/github/PinkPigmyPuff/SeatingChart/blob/main/ClassroomGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
from typing import List, Optional, Callable, Tuple


Chromosome = List[int]
Population = List[Chromosome]
Opinion = List[int]

In [2]:
def genChromosome(size: int) -> Chromosome:
    return random.sample(range(0, size), size)

def genPopulation(popSize: int, chromeosomeSize: int) -> Population:
    return [genChromosome(chromeosomeSize) for _ in range(popSize)]

def genOpinions(chromosome: Chromosome, opinionMax: int) -> Opinion:
  likes = []
  dislikes = []
  # for every student, randomly generate how many students they have opionions on, and add an empty array to like/dislike
  for i in range(0, len(chromosome)):
    likes.append([])
    dislikes.append([])
    likeNum = random.randint(0, opinionMax)
    dislikeNum = random.randint(0, opinionMax)
    dupelist = chromosome.copy()
    dupelist.pop(i)
    
    # choose random students that the current student likes
    for j in range(0, likeNum):
      randomStudent = random.randrange(0, len(dupelist))
      likes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

    # choose random students that the current student dislikes
    for j in range(0, dislikeNum):
      randomStudent = random.randrange(0, len(dupelist))
      dislikes[i].append(dupelist[randomStudent])
      dupelist.pop(randomStudent)

  return likes, dislikes

In [3]:
def fitness(chromosome: Chromosome, peoplePerTable: int = 4, nextToFriendCost: int = 2, nextToEnemyCost: int = -1) -> int:
    size = len(chromosome)
    if size != len(likes) != len(dislikes):
        raise ValueError("genome and likes/dislikes must be of the same length")

    if size % peoplePerTable != 0:
        raise ValueError("The length of the chromosome must be divisable by peoplePerTable")
    # determine the cost of the given chromosome
    peopleHappiness = [0] * size
    chunkedClassroom = chunks(chromosome, peoplePerTable)
    # for every student
    for student in range(0, len(chromosome)):
      currentStudent = chromosome[student]
      studentsTable = 0
      # finds which table the currentStudent is sitting at
      if any(currentStudent in (match := nested_list) for nested_list in chunkedClassroom):
        studentsTable = chunkedClassroom.index(match)
        # print(f"current student is {currentStudent}, and they sit at table {studentsTable}")

      # determine the happiness of the currentStudent, based on each person at their table
      for tableMate in range(0, peoplePerTable):
        currentTableMate = chunkedClassroom[studentsTable][tableMate]
        if currentTableMate in likes[student]:
          peopleHappiness[student] += nextToFriendCost
        elif currentTableMate in dislikes[student]:
          peopleHappiness[student] += nextToEnemyCost

    return average(peopleHappiness)

def populationFitness(population: Population) -> float:
    return [fitness(chromosome) for chromosome in population]
    
# Return n-sized chunks from lst
def chunks(lst, n):
    x = [lst[i:i + n] for i in range(0, len(lst), n)]
    return(x)

def relu(x):
	  return max(0.0, x)
 
def average(lst):
   return float(sum(lst) / len(lst))

In [4]:
# Randomly chooses n individuals from the population and returns the fittest one
def tournamentSelection(population: Population, tournamentSize: int) -> Population:
		
		tournament = []
		for i in range(0, tournamentSize):
			random_id = random.randint(0, len(population)-1)
			tournament.append(population[random_id])


		#print(f"tournament pool! {tournament}")
		fittest = chooseHighest(tournament, 1)
		#print(f"fittest! {fittest}")
		return fittest

In [5]:
def roulette_wheel_selection(population: Population, populationFitness) -> Population:
    if len(population) != len(populationFitness):
        raise ValueError("genome and likes/dislikes must be of the same length")
    print(f"pFit: {populationFitness}")

    # Computes the totallity of the population fitness
    sumFitness = sum(populationFitness)
    print(f"sumFittness: {sumFitness}")
    
    # Computes for each chromosome the probability 
    chromosome_probabilities = [classroomHappiness/sumFitness for classroomHappiness in populationFitness]
    print(f"c_prob {chromosome_probabilities}")

    # Selects one chromosome based on the computed probabilities
    choice = population[np.random.choice(range(0, len(population)), p=chromosome_probabilities)]
    print(f"chioce_fitness {fitness(choice)}")
    return choice

In [6]:
def partiallyMatchedCrossover(ind1, ind2):
    #print(f"ind1: {ind1}\nind2: {ind2}")
    size = min(len(ind1), len(ind2))
    p1, p2 = [0] * size, [0] * size

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[ind1[i]] = i
        p2[ind2[i]] = i
    # Choose crossover points
    cxpoint1 = random.randint(0, size)
    cxpoint2 = random.randint(0, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else:  # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    # Apply crossover between cx points
    for i in range(cxpoint1, cxpoint2):
        # Keep track of the selected values
        temp1 = ind1[i]
        temp2 = ind2[i]
        # Swap the matched value
        ind1[i], ind1[p1[temp2]] = temp2, temp1
        ind2[i], ind2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    #print(f"FINSIHED, ind1: {ind1}\n ind2: {ind2}")
    return ind1, ind2

In [7]:
def mutate(chromosome: Chromosome) -> Chromosome:
    length = len(chromosome)
    if length < 2:
      print("Chromosome is too short to be mutated")
      return chromosome
    index1, index2 = random.sample(range(0, length), 2)
    chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
    return chromosome

In [8]:
def chooseHighest(population: Population, num: int = 3):
  print(f"pop to choose from: {population}")
  fit = populationFitness(population)
  print(f"pop fitness: {fit}")
  highestIndex = 0
  for i in range(0, len(fit)):
      if fit[i] > fit[highestIndex]:
          highest = fit[i]
          highestIndex = i
  print(f"chosen: {population[highestIndex]}")
  return population[highestIndex]

In [9]:
# Generates an entirely new parent population, and shuffles it
def newPop(breedingPop: Population) -> Population:
  newPop = []
  for i in range(len(breedingPop)):
      offspring = tournamentSelection(breedingPop, 4)
      newPop.append(mutate(offspring))
  return newPop

In [10]:
pop = genPopulation(8, 8)
likes, dislikes = genOpinions(pop[0], 3)

def run_evolution(
        generation_limit: int = 30,
        population = genPopulation(8, 8), 
        #likes = genOpinions(population[0], 4),
        #dislikes = genOpinions(population[0], 4)
        #peoplePerTable = 4,
        #nextToFriendCost = 1,
        #nextToEnemyCost = -1,
        ):
    print(f"OG population: {population}")
    print(f"Likes: {likes}\nDislikes: {dislikes}")
    if len(population) % 2 != 0:
        print("Population size is odd, removing one chromosome")
        population.pop(0)


    for i in range(generation_limit):
        #random.shuffle(population)
        #parents = [tournamentSelection(population, 2) for i in range(len(population))]
        popFitness = populationFitness(population)
        print(f"\n\nGeneration {i} fitness: {popFitness}")
        print(f"population: {population}")
        ##parents = [roulette_wheel_selection(population, popFitness) for i in range(len(population))]

        population = newPop(population)

        #while parents:
            #print(f"parents: {parents}")
            #offspring1, offspring2 = partiallyMatchedCrossover(parents[0], parents[1])
            #print(f"nextPopulation CROSSED: {offspring1, offspring2}")
            #next_generation.append(mutate(offspring1))
            #next_generation.append(mutate(offspring2))
            #print(f"nextPopulation MUTATED: {next_generation}")
            #parents = parents[2:]
            
        
        #population = next_generation

    return chooseHighest(population)

run_evolution(population=pop)

OG population: [[3, 4, 5, 1, 2, 6, 0, 7], [4, 3, 5, 6, 2, 1, 7, 0], [4, 1, 6, 3, 7, 5, 2, 0], [5, 3, 7, 4, 0, 6, 1, 2], [4, 2, 5, 0, 3, 6, 7, 1], [5, 3, 1, 7, 0, 4, 2, 6], [1, 7, 2, 6, 3, 0, 5, 4], [0, 2, 3, 4, 1, 6, 7, 5]]
Likes: [[], [], [2, 7, 6], [0], [], [7], [3, 7], []]
Dislikes: [[6, 2], [], [], [], [], [], [6, 5, 1], [5, 6]]


Generation 0 fitness: [0.25, 0.5, 0.375, -0.125, 0.75, 0.0, 0.5, 0.25]
population: [[3, 4, 5, 1, 2, 6, 0, 7], [4, 3, 5, 6, 2, 1, 7, 0], [4, 1, 6, 3, 7, 5, 2, 0], [5, 3, 7, 4, 0, 6, 1, 2], [4, 2, 5, 0, 3, 6, 7, 1], [5, 3, 1, 7, 0, 4, 2, 6], [1, 7, 2, 6, 3, 0, 5, 4], [0, 2, 3, 4, 1, 6, 7, 5]]
pop to choose from: [[5, 3, 7, 4, 0, 6, 1, 2], [4, 2, 5, 0, 3, 6, 7, 1], [1, 7, 2, 6, 3, 0, 5, 4], [5, 3, 1, 7, 0, 4, 2, 6]]
pop fitness: [-0.125, 0.75, 0.5, 0.0]
chosen: [4, 2, 5, 0, 3, 6, 7, 1]
pop to choose from: [[4, 1, 6, 3, 7, 5, 2, 0], [5, 3, 7, 4, 0, 6, 1, 2], [1, 7, 2, 6, 3, 0, 5, 4], [4, 2, 5, 0, 3, 6, 7, 1]]
pop fitness: [0.375, -0.125, 0.5, 0.75]
chosen: [4

[4, 2, 5, 0, 3, 6, 7, 1]

1. Encoding scheme
- A chromosome is a permutation list
- Each index is an integer
- Each integer is a student in the class

2. Create the initial population
- First, a function to create a single chromsome
  - inputs chromosomeSize
  - outputs a chromosome of randomly shuffled integers

7. Testing
- A func